In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [18]:
path = 'data/statefarm/'
model_path = path + 'model/'

In [19]:
model = vgg_ft(10)

In [6]:
model.load_weights(model_path+'fintune6.h5')

In [20]:
last_conv_index = [index for index,layer in enumerate(model.layers)
                    if type(layer) == Convolution2D][-1]

In [21]:
last_conv_index

30

In [22]:
conv_model = [layer for layer in model.layers[:last_conv_index + 1]]

In [23]:
conv_model = Sequential(conv_model)

In [24]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
                                                                   zeropadding2d_14[1][0]           
___________________________________________________________________________________________

In [25]:
#now getting the fully connected layers ( used for copying weights so that we dont have to train the later layers from scratch)
fc_model = Sequential(model.layers[last_conv_index+1:])

In [26]:
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_10 (MaxPooling2D)   (None, 512, 7, 7)     0           convolution2d_26[0][0]           
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 25088)         0           maxpooling2d_10[0][0]            
                                                                   maxpooling2d_10[0][0]            
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 4096)          0           flatten_2[0][0]                  
                                                                   flatten_2[1][0]                  
___________________________________________________________________________________________

In [29]:
#Getting the batches 
batches = get_batches(path+'train/', batch_size=64)
val_batches = get_batches(path+'valid/', batch_size=64)

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [30]:
#onehot encoding the labels
train_classes = batches.classes
val_classes = val_batches.classes
train_labels = onehot(train_classes)
val_labels = onehot(val_classes)

In [32]:
train_features = conv_model.predict_generator(batches, batches.N)

In [34]:
save_array(model_path+'conv_train_features.bc', train_features)

In [ ]:
#ToDo next : Test model with all the denser layers ( train for say 5 epochs)
#            Test model with last "relu" dense layer removed 
#            Test model with second last relu layer removed
# Maybe : Test model with just the conv layers and a final softmax layer ( just to see possible drop in accuracy as feautre extraction { dense} layers play a big role)